In [1]:
import xmlrpc.client
import pandas as pd
from datetime import datetime
import time

# ------------------------------------------------------------
# 1) Conexión a Odoo
# ------------------------------------------------------------
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


print("\n🔌 Conectado a Odoo\n")


# ----------------------------------------------------------
# 3) DEFINIR FILTROS EXACTOS QUE ME DISTE
# ----------------------------------------------------------
domain = [
    ("date", ">=", "2025-01-01"),
    ("date", "<=", "2025-12-31"),
    ("type", "in", ["out_invoice", "out_refund"]),
    ("state", "=", "open"),
    ("commercial_partner_id", "!=", False),
]

print("Aplicando dominio:", domain)
print("Buscando IDs...\n")

# ----------------------------------------------------------
# 4) Buscar IDs primero (RÁPIDO) — así mostramos cuántos son
# ----------------------------------------------------------
ids = models.execute_kw(db, uid, password,
    "account.invoice.report",
    "search",
    [domain]
)

total_ids = len(ids)
print(f"📌 Registros encontrados: {total_ids}\n")

if total_ids == 0:
    print("No hay registros para descargar.")
    raise SystemExit()


# ----------------------------------------------------------
# 5) Leer datos en batches con PROGRESO
# ----------------------------------------------------------
batch = 5000
all_rows = []
fields = [
    "id", "date", "partner_id", "commercial_partner_id",
    "product_id", "product_qty", "price_total",
    "product_cost", "user_id", "state", "type", "product_marca",
    "section_id", "state_id","display_name"
]

print("📥 Descargando datos por bloques...\n")
start_time = time.time()

for i in range(0, total_ids, batch):
    chunk_ids = ids[i:i+batch]
    rows = models.execute_kw(db, uid, password,
        "account.invoice.report",
        "read",
        [chunk_ids],
        {"fields": fields}
    )

    all_rows.extend(rows)

    # Mostrar progreso
    percent = (i + len(chunk_ids)) / total_ids * 100
    print(f" → {i + len(chunk_ids)} / {total_ids} ({percent:.2f}%) descargados...")

print("\n✅ Descarga completa.")
print(f"⏱️ Tiempo total: {time.time() - start_time:.2f} segundos\n")


# ----------------------------------------------------------
# 6) Convertir a DataFrame
# ----------------------------------------------------------
df = pd.DataFrame(all_rows)
print("📊 DataFrame generado con", len(df), "filas")





🔌 Conectado a Odoo

Aplicando dominio: [('date', '>=', '2025-01-01'), ('date', '<=', '2025-12-31'), ('type', 'in', ['out_invoice', 'out_refund']), ('state', '=', 'open'), ('commercial_partner_id', '!=', False)]
Buscando IDs...

📌 Registros encontrados: 10525

📥 Descargando datos por bloques...

 → 5000 / 10525 (47.51%) descargados...
 → 10000 / 10525 (95.01%) descargados...
 → 10525 / 10525 (100.00%) descargados...

✅ Descarga completa.
⏱️ Tiempo total: 199.80 segundos

📊 DataFrame generado con 10525 filas


In [82]:
df["mes"].describe()

count    6725.000000
mean        8.409219
std         2.855998
min         1.000000
25%         9.000000
50%        10.000000
75%        10.000000
max        10.000000
Name: mes, dtype: float64

In [3]:
import pandas as pd
import re

df_clean = df.copy()

# -----------------------------------------------------------
# 1) PRODUCT_REF y PRODUCT_NAME
# product_id = [ID, [REF] NAME COMPLETO]
# -----------------------------------------------------------
def extract_product_ref(val):
    if isinstance(val, list) and len(val) >= 2:
        txt = val[1]
        m = re.search(r"\[([^\]]+)\]", txt)
        return m.group(1) if m else None
    return None

def extract_product_name(val):
    if isinstance(val, list) and len(val) >= 2:
        txt = val[1]
        return re.sub(r"\[[^\]]+\]\s*", "", txt).strip()
    return None

df_clean["product_ref"] = df_clean["product_id"].apply(extract_product_ref)
df_clean["product_name"] = df_clean["product_id"].apply(extract_product_name)


# -----------------------------------------------------------
# 2) SELLER NAME desde user_id
# user_id = [289, NOMBRE VENDEDOR]
# -----------------------------------------------------------
def extract_seller_name(val):
    if isinstance(val, list) and len(val) >= 2:
        return val[1].strip()
    return None

df_clean["seller_name"] = df_clean["user_id"].apply(extract_seller_name)

# Eliminar vendedores administrador
df_clean = df_clean[df_clean["seller_name"].str.lower() != "administrador administrador"]


# -----------------------------------------------------------
# 3) SECTION_ID  → store_name + tipo_venta
# Ejemplo:
# [14, "SUCURSALES / Ventas Credito Valladolid"]
# → tipo_venta = SUCURSALES
# → store_name = Ventas Credito Valladolid
# -----------------------------------------------------------
def extract_tipo_venta(val):
    if isinstance(val, list) and len(val) >= 2:
        txt = val[1]
        return txt.split("/")[0].strip()
    return None

def extract_store_name(val):
    if isinstance(val, list) and len(val) >= 2:
        txt = val[1]
        parts = txt.split("/")
        if len(parts) > 1:
            return parts[-1].strip()
    return None

df_clean["tipo_venta"] = df_clean["section_id"].apply(extract_tipo_venta)
df_clean["store_name"] = df_clean["section_id"].apply(extract_store_name)


# -----------------------------------------------------------
# 4) partner_name (cliente) desde partner_id
# -----------------------------------------------------------
def extract_partner_name(val):
    if isinstance(val, list) and len(val) >= 2:
        txt = val[1]
        return re.sub(r"\[[^\]]+\]\s*", "", txt).strip()
    return None

df_clean["partner_name"] = df_clean["partner_id"].apply(extract_partner_name)


# -----------------------------------------------------------
# 5) country_name desde country_id
# -----------------------------------------------------------
def extract_country(val):
    if isinstance(val, list) and len(val) >= 2:
        return val[1]
    return None




# -----------------------------------------------------------
# 6) Descripción de qué es NBR
# -----------------------------------------------------------
# NBR = número de líneas de factura (Odoo Invoice Report)
# Es una métrica interna: cantidad de líneas que componen la factura.
# No se transforma.


# -----------------------------------------------------------
# 7) Dejar solo columnas finales limpias
# -----------------------------------------------------------
df_final = df_clean[[
    "date",
    "product_ref",
    "product_name",
    "product_qty",
    "price_total",
    "product_cost",
    "seller_name",
    "tipo_venta",
    "store_name",
    "partner_name",
    "product_marca",
    "state",
    "type", "state_id"
]]

# Convertir a datetime
df_final["date"] = pd.to_datetime(df_final["date"], errors="coerce")

# Crear año
df_final["Año"] = df_final["date"].dt.year

# Crear mes numérico
df_final["Mes"] = df_final["date"].dt.month




df = df_final.copy()

/tmp/ipykernel_41397/3893592475.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["date"] = pd.to_datetime(df_final["date"], errors="coerce")
/tmp/ipykernel_41397/3893592475.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Año"] = df_final["date"].dt.year
/tmp/ipykernel_41397/3893592475.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [4]:
#df = pd.read_excel("/home/donsson/proyectos/VENTAS/datasets_salida/ventasxmesxvendedor.xlsx")

In [5]:
mapeo = {
    "DONSSON": "DONSSON",
    "BALDWIN": "BALDWIN",
    "RACOR BRASIL": "PARKER",
    "RACOR USA": "PARKER",
    "DELO": "ACEITE",
    "MOBIL IMPORTADO": "ACEITE",
    "MOBIL NACIONAL": "ACEITE",
    "KOMATSU": "ACEITE",
    "PURE GUARD": "ACEITE",
    "URSA": "ACEITE",
    "SUPREME": "ACEITE",
    "HIDROCOOL": "REFRIGERANTES",
    "TEKCOOL": "REFRIGERANTES",
    "OTROS": "OTROS",
    "False": "OTROS",
    "Indefinido": "OTROS",
    "RAMA": "OTROS_FILTROS",
    "PERKINS": "OTROS_FILTROS",
    "AUT*PARTS": "AUT*PARTS"
}

df["marca_agrupada"] = df["product_marca"].map(mapeo).fillna("OTROS")

# Convertir a datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Crear año
df["año"] = df["date"].dt.year

# Crear mes numérico
df["mes"] = df["date"].dt.month

In [6]:
df.head()
#df.to_excel("/home/donsson/proyectos/SIMON/exports/muestra.xlsx")

,date,product_ref,product_name,product_qty,price_total,product_cost,seller_name,tipo_venta,store_name,partner_name,product_marca,state,type,state_id,Año,Mes,marca_agrupada,año,mes
0,2025-11-21,BAE09107125,DA9207 FILTRO AIRE SEGURIDAD- BOBCAT (125 PA31...,1.0,91200.0,56134.00,Camila Arguello,SUCURSALES,Ventas Mostrador Cll6,NELSON QUERUBIN MARTIN DIAZ,BALDWIN,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,11,BALDWIN,2025,11
1,2025-11-21,DAB04869025,DA4869 FILTRO AIRE SEGURIDAD CAT. (025 DA4869),1.0,39000.0,12600.97,NEYDER CRESPO YEPES,SUCURSALES,Ventas mostrador Barranquilla,CONSTRUCCIONES MOLINOS OCHOA SAS,DONSSON,open,out_invoice,"[223, [08573] Atlántico / Puerto Colombia]",2025,11,DONSSON,2025,11
2,2025-11-21,DCE01005137,G1005 FILTRO COMBUSTIBLE SEPARADOR - VOLKSWAG...,3.0,114900.0,76300.74,WILSON PELAEZ,SUCURSALES,EXTERNOS MED,ANTIOQUEÑA DE LUBRICANTES SGP SAS,RACOR BRASIL,open,out_invoice,"[85, [05001] Antioquia / Medellín]",2025,11,PARKER,2025,11
3,2025-11-21,DCS00540189,GS540 FILTRO COMBUSTIBLE - CUMMINS (189 GS540),1.0,56400.0,19455.22,Camila Arguello,SUCURSALES,Ventas Mostrador Cll6,MARELVIS MEJIA CAMACHO,AUT*PARTS,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,11,AUT*PARTS,2025,11
4,2025-11-21,BCS00180125,GS180 FILTRO COMBUSTIBLE SEPARADOR KODIAK (ALT...,3.0,180300.0,120353.04,WILMER GIL,EXTERNOS,None,MARIA VICTORIA VARON PRADA,BALDWIN,open,out_invoice,"[377, [15759] Boyacá / Sogamoso]",2025,11,BALDWIN,2025,11


In [7]:
df["marca_agrupada"].unique()

array(['BALDWIN', 'DONSSON', 'PARKER', 'AUT*PARTS', 'OTROS', 'ACEITE',
       'REFRIGERANTES'], dtype=object)

In [8]:
df = df[df["date"]<"2025-11-1"]

In [9]:
df["seller_name"] = (
    df["seller_name"]
    .str.strip()          # quita espacios al inicio y final
    .str.upper()          # pasa a MAYÚSCULAS
    .str.normalize("NFKD")# elimina tildes/acentos (si quieres)
)


df["date"].max()

Timestamp('2025-10-31 00:00:00')

In [10]:
df["seller_name"].unique()

array(['ALEXANDRA URRIBARRI', 'ANDRES CORRALES',
       'EDWIN FERNANDO COLLAZOS BARRETO', 'WILSON PELAEZ',
       'JUAN CARLOS BETANCOURT GUTIERREZ', 'NEYDER CRESPO YEPES',
       'OMAR HERNANDEZ', 'JESUS ADONAY GONZALEZ FLOREZ',
       'NATALI ESPINAL TRUJILLO', 'GERMAN  RICARDO BETANCOURT GUTIERREZ',
       'WILMER GIL', 'HAROLD JIMENEZ', 'LUIS IGNACIO ARANGO PINZON',
       'VICTOR ALEJANDRO GONZALEZ RODRIGUEZ', 'PATRICIA NUÑEZ VALENCIA',
       'SANDRA RAMIREZ', 'LUIS EDUARDO ARANGO', 'GERMAN MORALES CARDENAS',
       'YEIS OSWALDO MUÑOZ GUERRERO', 'JAIME ALBERTO CAÑON RAMIREZ',
       'EDISSON MIGUEL CIFUENTES OSORIO', 'FABIAN BLANCO',
       'HUGO ALEJANDRO ZUBIETA FERNANDEZ',
       'ANDRES FELIPE ORDOÑEZ GREGORY', 'JUAN DAVID OCHOA',
       'MIGUEL ANGEL NIEBLES CERA', 'ISAAC SANCHEZ MONTAÑO',
       'MAICOL MORALES RAMIREZ'], dtype=object)

In [16]:
mostradores = df[df["tipo_venta"]=="SUCURSALES"]
externos = df[df["tipo_venta"]=="EXTERNOS"]

#mos = mostradores.drop(columns=["Unnamed: 0"])
#ex = externos.drop(columns=["Unnamed: 0"])

In [66]:
mostradores.sample(10)

,date,product_ref,product_name,product_qty,price_total,product_cost,seller_name,tipo_venta,store_name,partner_name,product_marca,state,type,state_id,Año,Mes,marca_agrupada,año,mes
4935,2025-10-24,DCE00046137,"G046 FILTRO COMBUSTIBLE PERKINS, MASSEY, OTROS...",5.0,119000.0,65339.15,JAIME ALBERTO CAÑON RAMIREZ,SUCURSALES,EXTERNOS,RAFAEL ESPINOSA G. & CIA. SAS,RACOR BRASIL,open,out_invoice,"[234, [13001] Bolivar / Cartagena]",2025,10,PARKER,2025,10
4392,2025-10-28,BCS00318125,GS318 FILTRO COMBUSTIBLE VOLVO (ALTA EFICIENCI...,2.0,86200.0,61941.48,FABIAN BLANCO,SUCURSALES,EXTERNOS BUCARAMANGA,RETOMAQ M&L S.A.S. BIC,BALDWIN,open,out_invoice,"[888, [54498] Norte de Santander / Ocaña]",2025,10,BALDWIN,2025,10
8679,2025-09-16,DAB08051025,DA8051 FILTRO AIRE FOTON OLIN BJ1039 5.8 TON. ...,6.0,306000.0,161758.26,YEIS OSWALDO MUÑOZ GUERRERO,SUCURSALES,Ventas Credito Norte,VISUM LTDA,DONSSON,open,out_invoice,"[647, [25843] Cundinamarca / Villa De San Dieg...",2025,9,DONSSON,2025,9
10196,2025-01-30,DCE00761137,G761 FILTRO COMBUSTIBLE PERKINS (137 REC817),1.0,28600.0,21947.09,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES,EXTERNOS,DIGABE DEL CARIBE SAS,RACOR BRASIL,open,out_invoice,"[210, [08001] Atlántico / Barranquilla]",2025,1,PARKER,2025,1
9940,2025-02-27,BLS00548125,GS548 FILTRO ACEITE INTERNATIONAL (125 BD7250),1.0,130500.0,103163.97,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES,EXTERNOS,DIGABE DEL CARIBE SAS,BALDWIN,open,out_invoice,"[210, [08001] Atlántico / Barranquilla]",2025,2,BALDWIN,2025,2
9722,2025-03-20,BCS00314125,GS314 FILTRO COMBUSTIBLE CUMMINS (SINTETICO) (...,2.0,102800.0,81552.34,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES,EXTERNOS,DIGABE DEL CARIBE SAS,BALDWIN,open,out_invoice,"[210, [08001] Atlántico / Barranquilla]",2025,3,BALDWIN,2025,3
6565,2025-10-14,DAB02982025,DA2982 FILTRO AIRE DONSSON - IHC 7600i MOTOR ...,1.0,123800.0,48076.36,WILMER GIL,SUCURSALES,EXTERNOS CALI,MARIO ALBERTO HUERTAS COTES,DONSSON,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,10,DONSSON,2025,10
5630,2025-10-21,DAB04982025,DA4982 FILTRO AIRE INT. CAMION IHC 7600 MOTOR ...,1.0,70700.0,24338.22,PATRICIA NUÑEZ VALENCIA,SUCURSALES,EXTERNOS CALI,MATERIALES Y AGREGADOS EMT S.A.S,DONSSON,open,out_invoice,"[1130, [76892] Valle del Cauca / Yumbo]",2025,10,DONSSON,2025,10
8602,2025-09-17,DAE01127189,GA1127 FILTRO AIRE TOYOTA HILUX 2.4 LTS. FORTU...,6.0,136200.0,45254.16,PATRICIA NUÑEZ VALENCIA,SUCURSALES,EXTERNOS CALI,INGENIERIA JOULES M.E.C. S.A.S,AUT*PARTS,open,out_invoice,"[689, [41001] Huila / Neiva]",2025,9,AUT*PARTS,2025,9
9435,2025-04-25,BCS00076125,GS076 FILTRO COMBUSTIBLE KOMATSU (125 BF941),2.0,58600.0,46791.74,GERMAN RICARDO BETANCOURT GUTIERREZ,SUCURSALES,EXTERNOS,DIGABE DEL CARIBE SAS,BALDWIN,open,out_invoice,"[210, [08001] Atlántico / Barranquilla]",2025,4,BALDWIN,2025,4


In [80]:
ex = externos.copy() 

ex.sample(10)

,date,product_ref,product_name,product_qty,price_total,product_cost,seller_name,tipo_venta,store_name,partner_name,product_marca,state,type,state_id,Año,Mes,marca_agrupada,año,mes
6450,2025-10-14,DAR08049025,"DA8049 FILTRO AIRE HYUNDAI HD78, HD65 (025 DA8...",12.0,655200.0,407773.44,JESUS ADONAY GONZALEZ FLOREZ,EXTERNOS,None,ORLANDO RATIVA ALBERTO,DONSSON,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,10,DONSSON,2025,10
9099,2025-09-01,DAS07801025,DA7801 FILTRO AIRE - CATERPILLAR - VOLVO - SDM...,12.0,724800.0,393440.76,LUIS IGNACIO ARANGO PINZON,EXTERNOS,None,LA CASA DEL FILTRO VILLAVICENCIO S.A.S.,DONSSON,open,out_invoice,"[771, [50001] Meta / Villavicencio]",2025,9,DONSSON,2025,9
6630,2025-10-14,BHE00009125,G009 FILTRO HIDRAULICO CATERPILLAR 9M9740. (12...,1.0,33000.0,25860.55,HUGO ALEJANDRO ZUBIETA FERNANDEZ,EXTERNOS,None,C.I. FILTERS S.A.S,BALDWIN,open,out_invoice,"[288, [15104] Boyacá / Boyacá]",2025,10,BALDWIN,2025,10
5133,2025-10-23,DAE04196025,"DA4196 FILTRO AIRE 2_ J.DEERE,FIAT,IHC. (025 D...",2.0,95200.0,43621.60,WILMER GIL,EXTERNOS,None,MARIO ALBERTO HUERTAS COTES,DONSSON,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,10,DONSSON,2025,10
8937,2025-09-11,DCS10168189,GS168A1 FILTRO COMBUSTIBLE SEPARADOR AGUA. AUT...,1.0,39000.0,17958.63,WILMER GIL,EXTERNOS,None,MARIO ALBERTO HUERTAS COTES,AUT*PARTS,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,9,AUT*PARTS,2025,9
7433,2025-10-08,BCS00691125,GS691 FILTRO COMBUSTIBLE JOHN DEERE (125 BF9891D),2.0,157000.0,119458.56,ANDRES CORRALES,EXTERNOS,None,OSCAR FERNANDO QUINO BONILLA,BALDWIN,open,out_invoice,"[689, [41001] Huila / Neiva]",2025,10,BALDWIN,2025,10
7090,2025-10-10,DAB02886025,DA2886 FILTRO AIRE- PERKINS (025 DA2886),1.0,36800.0,9854.47,WILMER GIL,EXTERNOS,None,MARIO ALBERTO HUERTAS COTES,DONSSON,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,10,DONSSON,2025,10
7884,2025-09-30,BCS00198125,"GS198 FILTRO COMBUSTIBLE HITACHI, MITSUBISHI. ...",1.0,58900.0,39045.08,LUIS IGNACIO ARANGO PINZON,EXTERNOS,None,AGROFILTER DEL LLANO S.A.S.,BALDWIN,open,out_invoice,"[772, [50006] Meta / Acacías]",2025,9,BALDWIN,2025,9
5935,2025-10-20,BLS00072125,GS072 FILTRO ACEITE DETROIT (125 B95),6.0,381600.0,282100.50,LUIS IGNACIO ARANGO PINZON,EXTERNOS,None,LOPEZ RAMIREZ CAMPO ELIAS,BALDWIN,open,out_invoice,"[771, [50001] Meta / Villavicencio]",2025,10,BALDWIN,2025,10
5899,2025-10-20,BLS00286125,GS286 FILTRO ACEITE DUAL MOTORES CUMMINS (125 ...,1.0,107200.0,87059.60,WILMER GIL,EXTERNOS,None,MARIO ALBERTO HUERTAS COTES,BALDWIN,open,out_invoice,"[233, [11001] Bogotá / Bogotá, D.C.]",2025,10,BALDWIN,2025,10


In [19]:
# Agrupar por vendedor, marca agrupada, mes y año
ex_resumen = (
    ex.groupby(["seller_name", "marca_agrupada", "mes", "año"])
      .agg(
          unidades_vendidas=("product_qty", "sum"),
          subtotal_mes=("price_total", "sum")
      )
      .reset_index()
      .sort_values(["seller_name", "marca_agrupada", "año", "mes"])
)

ex_resumen


ex_pivot = ex_resumen.pivot_table(
    index=["seller_name", "marca_agrupada"],
    columns=["mes"],
    values=["unidades_vendidas", "subtotal_mes"],
    fill_value=0
)

ex_pivot.head(30)


subtotal_mes             \
mes                                                       2          6    
seller_name                      marca_agrupada                           
ALEXANDRA URRIBARRI              DONSSON            -50300.0        0.0   
ANDRES CORRALES                  AUT*PARTS               0.0        0.0   
                                 BALDWIN                 0.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 OTROS                   0.0        0.0   
                                 PARKER                  0.0        0.0   
HAROLD JIMENEZ                   AUT*PARTS               0.0    40400.0   
                                 BALDWIN                 0.0  2182600.0   
                                 DONSSON                 0.0  1302000.0   
                                 PARKER                  0.0        0.0   
HUGO ALEJANDRO ZUBIETA FERNANDEZ AUT*PARTS               0.0        0.0   
                                 BALDWIN            -38300.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 PARKER                  0.0        0.0   
ISAAC SANCHEZ MONTAÑO           AUT*PARTS               0.0        0.0   
                                 BALDWIN                 0.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 PARKER                  0.0        0.0   
JESUS ADONAY GONZALEZ FLOREZ     AUT*PARTS               0.0        0.0   
                                 BALDWIN                 0.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 OTROS                   0.0        0.0   
                                 PARKER                  0.0        0.0   
JUAN DAVID OCHOA                 BALDWIN                 0.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 PARKER                  0.0        0.0   
LUIS EDUARDO ARANGO              AUT*PARTS               0.0        0.0   
                                 BALDWIN                 0.0        0.0   
                                 DONSSON                 0.0        0.0   
                                 OTROS                   0.0        0.0   

                                                                        \
mes                                                     7           8    
seller_name                      marca_agrupada                          
ALEXANDRA URRIBARRI              DONSSON               0.0         0.0   
ANDRES CORRALES                  AUT*PARTS             0.0         0.0   
                                 BALDWIN         3587300.0         0.0   
                                 DONSSON         1524100.0         0.0   
                                 OTROS                 0.0         0.0   
                                 PARKER                0.0         0.0   
HAROLD JIMENEZ                   AUT*PARTS             0.0         0.0   
                                 BALDWIN               0.0   5185900.0   
                                 DONSSON               0.0   1812000.0   
                                 PARKER                0.0    425000.0   
HUGO ALEJANDRO ZUBIETA FERNANDEZ AUT*PARTS             0.0   6516000.0   
                                 BALDWIN               0.0         0.0   
                                 DONSSON               0.0    752000.0   
                                 PARKER                0.0         0.0   
ISAAC SANCHEZ MONTAÑO           AUT*PARTS             0.0         0.0   
                                 BALDWIN               0.0         0.0   
                                 DONSSON               0.0         0.0   
                                 PARKER                0.0         0.0   
JESUS ADONAY GONZA

In [ ]:
# Agrupar por vendedor, marca agrupada, mes y año
mos_resumen = (
    mos.groupby(["store_name", "marca_agrupada", "mes", "año"])
      .agg(
          unidades_vendidas=("quantity", "sum"),
          subtotal_mes=("price_subtotal", "sum")
      )
      .reset_index()
      .sort_values(["store_name", "marca_agrupada", "año", "mes"])
)




mos_pivot = mos_resumen.pivot_table(
    index=["store_name", "marca_agrupada"],
    columns=["mes"],
    values=["unidades_vendidas", "subtotal_mes"],
    fill_value=0
)

mos_pivot

In [ ]:
def flatten_columns(df):
    df = df.copy()
    df.columns = [
        "_".join([str(level) for level in col]).strip("_")
        if isinstance(col, tuple)
        else col
        for col in df.columns
    ]
    return df.reset_index()

mos_export = flatten_columns(mos_pivot)
ex_export = flatten_columns(ex_pivot)

with pd.ExcelWriter("/home/donsson/proyectos/SIMON/exports/reporte_pivots.xlsx") as writer:
    mos_export.to_excel(writer, sheet_name="mos_pivot", index=False)
    ex_export.to_excel(writer, sheet_name="ex_pivot", index=False)
